# miRNA-Seq

## Validation by Real-Time PCR

On this experiment, I have tested the amplification of a set of miRNAS (detailed below) on [Female EF sample (RNA 50)](qRT-PCR_optimizations.ipynb).

### List of miRNAs
* vvi-mir156(d) (positive control, see below)
* vvi-mir160(d) (may not work, see below)
* vvi-mir168
* vvi-mir172(b)
* vii-mir828(a) (may not work, see below)

#### Notes
* vvi-mir156(d) was used as positive control, as this primer already amplified [previously](qRT-PCR_optimizations.ipynb). Also, I wanted to test the consequences of duplicating the Evagreen quantity, so this test will evaluate that.

* vvi-mir160(b) and vvi-mir828(a) may not work as these miRNAs are not expected to be present in this sample.

* Test was performed in duplicates.

* Primer R was Oligo(dT).


### Protocol

| Component | 1X (µL) | 11X (µL) |
| :-------- |-------: | -------: |
| Evagreen  |    10.0 |    110.0 |
| cDNA      |     0.5 |      5.5 |
| Primer F  |     0.5 |      --- |
| Primer R  |     0.5 |      5.5 |
| H20       |     8.5 |     93.5 |


### Plate Setup

* B2-3: vvi-mir168
* B4-5: vvi-mir172
* C2-3: vvi-mir160 (?)
* C4-5: vvi-mir828
* D2-3: vvi-mir156 (?)

### Loading required modules

In [ ]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt

path = '20/'

### Preparing data

In [ ]:
columns = {
    'Well/Cycles': 'cycle',
    
    'B02': 'mir168_r1', 'B03': 'mir168_r2',
    'B04': 'mir172_r1', 'B05': 'mir172_r2',
        
    'C02': 'mir160_r1', 'C03': 'mir160_r2',
    'C04': 'mir828_r1', 'C05': 'mir828_r2',
    
    'D02': 'mir156_r1', 'D03': 'mir156_r2'
}

### Open amplification file

In [ ]:
amplification = pd.read_csv(f'{path}amplification_data_rfu.tsv',
                            sep = '\t',
                            header = 0,
                            index_col = 0
                           )

amplification = amplification.rename(columns = columns)
amplification.head()

### Verifing amplification curves

In [ ]:
columns_list = list(columns.values())

# The first item will be cycle, with is not relevant
columns_list.pop(0)

# Remove reaction number
columns_list = list(map(lambda x: x.rsplit('_', 1)[0], columns_list))

# Remove the duplicates
columns_list = list(set(columns_list))

# Plot the amplifications
for sample in columns_list:
    amplification[[sample + '_r1', sample + '_r2']].plot()

### Considerations on amplification

The amplifications are not exactly as expected from miRNA-Seq:
* vvi-mir156 had amplification on only one of the reactions. Both should had work (success: 1/2)
* vvi-mir160 was not expected to express, and no expression was found (success: 2/2 | 3/4)
* vvi-mir168 was expected to express, but no expression was found (success: 0/2 | 3/6)
* vvi-mir172 was expected to express and in fact, it express (success: 2/2 | 5/8)
* vvi-mir828 is a negative control, but we find expression in one reaction (success: 1/2 | 6/9)

### Opening melting file

In [ ]:
melting = pd.read_csv(f'{path}melting_data_rfu.tsv',
                      sep = '\t',
                      header = 0,
                      index_col = 0
                     )

melting = melting.rename(columns = columns)
melting.head()

#### Calculating the melting derivate

The melting derivate will be calculated according to the formula:
`-delta(RFU) / delta(temperature)`

On this case, the difference in temperature is always 0.5 as that was the parameters set on the experiment.

In [ ]:
def derivate_melting(values):
    delta_value = []
    for i in range(1, len(values)):
        j = i - 1
        delta_value.append(values[i] - values[j])
    
    derivate = list(map(lambda x: -x/0.5, delta_value))
    return(derivate)

In [ ]:
for sample in columns_list:
    derivate_1 = derivate_melting(melting[sample + '_r1'].values.tolist())
    plt.plot(derivate_1)
    
    derivate_2 = derivate_melting(melting[sample + '_r2'].values.tolist())    
    plt.plot(derivate_2)
    
    plt.title(sample)
    
    plt.show()

### Considerations on melting curve

All primers have specific amplifications

### Global conclusions

I have made a mistaken, by choosing RNA50. **I should have chosen RNA5**, which was the RNA sample that was diluted 1/5. I need to repeat this RT-qPCR to confirm the amplifications.

Nevertheless, there may exists problems in the primers for mir168 and mir828.

I am still not convinced by the mir156: It does not make sense to amplify on only one reaction (the same was verified [previously](qRT-PCR_optimizations.ipynb)).

The usage of 10 µL of evagreen was a good call. I will keep it.